In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
from mgwr.summary import summaryModel, summaryGLM, summaryGWR
from os import path
pd.set_option('display.max_columns', None)

/Users/dylanhalpern/miniconda/envs/pb_can_39/lib/python3.9/site-packages/libpysal/cg/alpha_shapes.py:39: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def nb_dist(x, y):
/Users/dylanhalpern/miniconda/envs/pb_can_39/lib/python3.9/site-packages/libpysal/cg/alpha_shapes.py:165: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_faces(triangle):
/Users/dylanhalpe

In [2]:
data_dir = path.join('..', 'data')

In [3]:
vision = pd.read_parquet(path.join(data_dir, "vision_providers_minimal.parquet"))
censusTractok = gpd.read_file(path.join(data_dir, "Oklahoma", "OK Tracts Collect.zip"))
##This dataframe contains information for ophthalmology only; Contains distances, geometry, and GEOID
oklahomamerged = pd.read_csv(path.join(data_dir, "correctdf.csv"))
okdemographs = pd.read_csv(path.join(data_dir, "Oklahoma","Socio-Econ_Factor_OK_by_Tract.csv"))

In [4]:
OKcentroids = censusTractok.copy().to_crs("ESRI:102008")
OKcentroids['geometry'] = OKcentroids['geometry'].centroid

In [6]:
#subsetting vision provider data based on classification (ie optometrist/ophthamologist) and entity type
#Filtering for vision care providers
vision2 = vision[vision["Entity Type Code"] == 1]

#Checking if taxonomy code starts with 152
vision2['Optometry'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('152') for code in x.split('|')))

#Checking if taxonomy code starts with 207
vision2['Ophthalmology'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('207') for code in x.split('|')))

#Checking if taxonomy code starts with 156
vision2['Others'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('156') for code in x.split('|')))

#Creating a GeoDataFrame with geometry from latitude and longitude
vision3 = gpd.GeoDataFrame(
   vision2, geometry=gpd.points_from_xy(vision2.Longitude, vision2.Latitude),
    crs="EPSG:4326")

#Filtering for providers in Oklahoma
vision3 = vision3[vision3["Provider Business Mailing Address State Name"] == 'OK']

#Changing the CRS to a projected CRS
vision3p = vision3.to_crs("ESRI:102008")

/var/folders/y0/hhpt053172987s509j7zft480000gn/T/ipykernel_60710/2026838144.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vision2['Optometry'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('152') for code in x.split('|')))
/var/folders/y0/hhpt053172987s509j7zft480000gn/T/ipykernel_60710/2026838144.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vision2['Ophthalmology'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('207') for code in x.split('|')))
/var/folders/y0/hhpt053172

In [7]:
merged_df = pd.merge(oklahomamerged, okdemographs, on='GEOID', how='inner')
merged_df['GEOID']=merged_df['GEOID'].astype(int)
censusTractok['GEOID']=censusTractok['GEOID'].astype(int)

In [8]:
zipmerge = pd.merge(censusTractok, merged_df, how= 'left', on= "GEOID")
zipmerge = zipmerge.set_geometry(zipmerge["geometry_x"])
y = zipmerge['Distances'].values.reshape((-1,1)) # reshape is needed to have column array
White = zipmerge['WhitePerc'].values.reshape((-1,1))

In [9]:
MHI = zipmerge['Median_household_income'].values.reshape((-1,1))
Bache = zipmerge['Bach_or_higher_perc'].values.reshape((-1,1))
X = np.array([MHI, Bache, White]).reshape((-1, 3))
X_shape = X.shape

In [10]:
u = zipmerge['Longitude']
v = zipmerge['Latitude']
coords = list(zip(u,v))

In [11]:
gwr_selector = Sel_BW(coords, y, X)
gwr_bw = gwr_selector.search()

In [12]:
gwr_bw = 118.0
gwr_results = GWR(coords, y, X, gwr_bw).fit()
gwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                1205
Number of covariates:                                                     4

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                       1227308953359.036
Log-likelihood:                                                  -14206.641
AIC:                                                              28421.282
AICc:                                                             28423.332
BIC:                                                           1227308944838.860
R2:                                                                   0.000
Adj. R2:                                                             -0.002

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- -------